In [1]:
# Usage guide:

# This document is not kept on record. If the calcualtions are to be kept,
# copy the whole notebook and put it in "quick_calculations" or appropriate
# folders.

In [2]:
# import all the E9 stuff
import logging
import numpy as np
import matplotlib.pyplot as plt
import sys
from pathlib import Path

# User defined modules
E9path = Path("C:/", "Users", "ken92", "Documents", "Studies", "E5", "simulation", "E9_simulations")
if str(E9path) not in sys.path:
    sys.path.insert(1, str(E9path))
import E9_fn.E9_constants as E9c
import E9_fn.E9_atom as E9a
import E9_fn.E9_cooltrap as E9ct
import E9_fn.polarizabilities_calculation as E9pol
# import E9_fn.datasets.transition_line_data as TLData
from E9_fn import util
from E9_fn.hardware.cameras import all_cameras

# Logging
logpath = '' # '' if not logging to a file
loglevel = logging.INFO
logroot = logging.getLogger()
list(map(logroot.removeHandler, logroot.handlers))
list(map(logroot.removeFilter, logroot.filters))
logging.basicConfig(filename = logpath, level = loglevel)

c:\Users\ken92\Documents\Studies\E5\simulation\E9_simulations\.venv\Lib\site-packages\gftool\precision.py:16: UserWarning: No quad precision datatypes available!
Some functions might be less accurate.
  warnings.warn("No quad precision datatypes available!\n"


In [3]:
# System size related
# Inputs
trap_freq_Hz = 30
N_all = 2e4               # includes all spin species
K39_a_s_a0 = 100          # interaction strength of K39, in units of a0
MSF_time = 10e-3          # Momentum space focusing time

trap_freq_rad = 2 * np.pi * trap_freq_Hz
K39_a_s = E9c.a0 * K39_a_s_a0
T_trap = 1 / trap_freq_Hz # Trap "period"
R_K39 = E9ct.fermi_radius(E9c.m_K39, trap_freq_rad, N_all, xi = -1, a_s = K39_a_s)
R_K40 = E9ct.fermi_radius(E9c.m_K40, trap_freq_rad, N_all/2, xi = 1)
d_ToF = (E9c.hbar * E9c.g_sw / E9c.m_K39) * T_trap
MSF_freq_rad = 2 * np.pi / (4 * 10e-3) # trap frequency of MSF B trap
print("Assuming (isotropic) trap frequency = 2*pi*{} Hz, total particle number = {}, T = 0".format(
    trap_freq_Hz, N_all))
print("K39: Fermi radius = {:.2f} um (scattering length = {} a0)".format(R_K39 * 1e6, K39_a_s_a0))
print("K40: Fermi radius = {:.2f} um (balanced spin-1/2 system)".format(R_K40 * 1e6))
print("K39 with momentum of one reciprocal lattice vector will fly\n" 
      "{:.2f} um in one trap period ({:.2f} ms)".format(d_ToF * 1e6, T_trap * 1e3))
print((E9c.hbar * E9c.g_sw / E9c.m_K39) / MSF_freq_rad *1e6)


Assuming (isotropic) trap frequency = 2*pi*30 Hz, total particle number = 20000.0, T = 0
K39: Fermi radius = 10.35 um (scattering length = 100 a0)
K40: Fermi radius = 25.69 um (balanced spin-1/2 system)
K39 with momentum of one reciprocal lattice vector will fly
1111.41 um in one trap period (33.33 ms)
212.2637170582222


In [4]:
# Mechanical constraints
ang = np.arctan(20.32/14.2)
NA = np.sin(ang)
print("Angle = {:.3f} deg, NA = {:.3f}".format(ang / np.pi * 180, NA))

Angle = 55.053 deg, NA = 0.820


In [5]:
# Resolution related
px_per_a_lat = 3
print(f"To map the lattice constant to {px_per_a_lat} pixel widths require:")
for cam in all_cameras:
    M_required = (cam.l_px / E9c.a_sw_tri) * px_per_a_lat
    FoV_um = 1e6 * cam.active_area / M_required
    print(f"--- {cam.name} ---")
    print(f"M = {M_required:.2f}")
    print(f"Field of view = {FoV_um[0]:.2f} x {FoV_um[1]:.2f} um")

To map the lattice constant to 3 pixel widths require:
--- iXon885 ---
M = 67.67
Field of view = 118.70 x 118.46 um
--- iXon888 ---
M = 109.96
Field of view = 121.06 x 121.06 um
--- iXon897 ---
M = 135.34
Field of view = 60.53 x 60.53 um
--- Marana11 ---
M = 93.05
Field of view = 242.12 x 242.12 um
--- Marana6 ---
M = 54.98
Field of view = 242.12 x 242.12 um
--- Stingray ---
M = 70.21
Field of view = 92.21 x 68.57 um
